# Experiments
Having implemented the different networks and models we can now run them and analyse the results of different parameterisations. This is meant as an exploratory analysis of the effects of different parameters. For the web app we will simualate the models with parameters on a finer grid.

In [27]:
%matplotlib inline

# allow relative imports
import os
import sys

module = os.path.abspath(os.path.join('..'))
if module not in sys.path:
    sys.path.append(module)
    

# utils
from functools import partial
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd

SEED = 4401

The following Human Contact Networks will be compared:
- Mobility network
- Power law with cutoff
- Physical distancing network

For each, we will evaluate the following epidemic models:
- SEIR
- SEIVR
- SEIR with quarantine
- SEVIR with quarantine

Each model will require slightly different parameters.

## Networks

For each of the three types of networks we will create two instances: a baseline network representing nodes living under no restrictions/distancing etc. (we will call this __pre__) and a network assuming some form of physical distancing put in place (called __post__). We can then plug in each of these networks into the different models.

The mobility networks have `10000` nodes and average node degrees of `4` and `3` for __pre__ and __post__ respectively. Where appropriate, we will mimick these characteristics in the other networks for comparability reasons.

TODO: Maybe only do pre and post for MN since others are completely arbitrary

### Mobility network

The mobility networks are based on mobility data from SafeGraph for New Haven County, CT, USA. The __pre__ network is adapted from data from February 2020, before stay-at-home orders were issued, while the __post__ network is based on data from April 2020, after such orders were put in place in March.

The graphs for the two networks have already been producen (see *network_creation.ipynb*), so we can use the `MNGeneratorFromFile`. For now, it suffices defining the parameters of the generator, which we can then pass to the experiment later on.

In [2]:
from configuration import GRAPHS, OUT
from model.network.mobility_network import MNGeneratorFromNetworkData as MNG
from data_processing.utils import load_network_data_from_files

First create the NetworkData instances that are required for all Mobility networks.

In [3]:
file_names = dict(
    demographics='demographics.pkl',
    comb_pre='comb_counts_pre.pkl',
    comb_post='comb_counts_post.pkl',
    trip_pre='trip_counts_pre.pkl',
    trip_post='trip_counts_post.pkl',
)

network_data = load_network_data_from_files(file_names, OUT)
network_data_pre = network_data['pre']
network_data_post = network_data['post']

In [4]:
params_mobility_pre = dict()
params_mobility_pre[MNG.EXPONENT] = 2
params_mobility_pre[MNG.CUTOFF] = 40
params_mobility_pre[MNG.N] = 10000
params_mobility_pre[MNG.NETWORK_DATA] = network_data_pre
params_mobility_pre[MNG.MULTIPLIER] = False

params_mobility_post = dict()
params_mobility_post[MNG.EXPONENT] = 2
params_mobility_post[MNG.CUTOFF] = 40
params_mobility_post[MNG.N] = 10000
params_mobility_post[MNG.NETWORK_DATA] = network_data_post
params_mobility_post[MNG.MULTIPLIER] = True

### Power law with cutoff

The power law with cutoff (PLC) creates a network with nodes with degrees that follow a power law distribution but have a cutoff point at the upper end to prevent implausibly high degrees. `epydemic` provides a NetworkGenerator for this type of network so we can create it from there. The cutoff parameter is set to `10`. Again, we only define the parameters now and pass them to the experiment with the generator later.

In [5]:
from epydemic import PLCNetwork

In [6]:
params_plc_pre = dict()
params_plc_pre[PLCNetwork.N] = 10000
params_plc_pre[PLCNetwork.EXPONENT] = 2
params_plc_pre[PLCNetwork.CUTOFF] = 40

params_plc_post = dict()
params_plc_post[PLCNetwork.N] = 10000
params_plc_post[PLCNetwork.EXPONENT] = 2
params_plc_post[PLCNetwork.CUTOFF] = 40

### Physical distancing

In [7]:
from model.network.distanced_network import DNGenerator as DNG
from model.distributions import discrete_trunc_normal, discrete_trunc_exponential, num_contact_dist

In [8]:
DISTANCED_EXPONENT_PRE = 4
DISTANCED_EXPONENT_POST = 3

household_size_dist = partial(discrete_trunc_normal, mu=4.5, std=2)

params_distanced_pre = dict()
params_distanced_pre[DNG.N] = 10000
params_distanced_pre[DNG.HOUSEHOLD_SIZE_DIST] = household_size_dist
params_distanced_pre[DNG.NUM_CONTACT_DIST] = num_contact_dist
params_distanced_pre[DNG.NUM_OUTSIDE_EDGE_DIST] = partial(discrete_trunc_exponential, exponent=DISTANCED_EXPONENT_PRE)

params_distanced_post = dict()
params_distanced_post[DNG.N] = 10000
params_distanced_post[DNG.HOUSEHOLD_SIZE_DIST] = household_size_dist
params_distanced_post[DNG.NUM_CONTACT_DIST] = num_contact_dist
params_distanced_post[DNG.NUM_OUTSIDE_EDGE_DIST] = partial(discrete_trunc_exponential, exponent=DISTANCED_EXPONENT_POST)

## Epidemic model

### SEIR

In [9]:
from epydemic import SEIR
from model.compartmental_models.seir import MonitoredSEIR

In [10]:
seir_compartments = {
    SEIR.SUSCEPTIBLE: 'susceptible',
    SEIR.EXPOSED: 'exposed',
    SEIR.INFECTED: 'infected',
    SEIR.REMOVED: 'removed'
}

### SEIR with quarantine

In [11]:
from model.compartmental_model.seir import SEIRWithQuarantine as SEIRQ
from model.compartmental_model.seir import MonitoredSEIRWithQuarantine as MonitoredSEIRQ

### SEIVR

In [12]:
from model.compartmental_model.seivr import SEIVR
from model.compartmental_model.seivr import MonitoredSEIVR

In [13]:
seivr_compartments = {
    SEIVR.SUSCEPTIBLE: 'susceptible',
    SEIVR.EXPOSED: 'exposed',
    SEIVR.INFECTED: 'infected',
    SEIVR.VACCINATED: 'vaccinated',
    SEIVR.REMOVED: 'removed'
}

### SEIVR with quarantine

In [14]:
from model.compartmental_model.seivr import SEIVRWithQuarantine as SEIVRQ
from model.compartmental_model.seivr import MonitoredSEIVRWithQuarantine as MonitoredSEIVRQ

## Parameters

We will fix as many COVID-19 specific parameters as we can and leave only policy dependent parameters open for modifications.

We set the following parameters

| Variable | Value | Description | Reference |
| :-- | --: | :-- | :-- |
| R0 | $2.85$ | Basic Reproduction Number | 1) |
| P_INFECT | ... | P. of infection | ... |
| P_REMOVE | $0.1$ | P. of removal | 2) |
| P_SYMPTOPMS | $0.2$ | P. of becoming symptomatic | 3) |
| P_EXPOSED | $0.01$ | P. of initial infection | ... |
| RRR | $0.95$, $0.67$ | Vaccine efficacy | 4), 5), 6) |
| P_VACCINATED | $10^{-3}$ to $10^{-2}$ by $10^{-3}$| P. of getting vaccinated | ... |
| P_VACCINATED_INITIAL | $0$ to $1$ by $0.1$ | P. of initial vaccination | ... |
| P_QUARANTINE | $0$ to $1$ by $0.1$ | P. of rewiring | ... |


1) https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0249271

2) https://www.thelancet.com/journals/lanmic/article/PIIS2666-5247(20)30172-5/fulltext

3) https://doi.org/10.3389/fpubh.2020.598547

4) https://www.nejm.org/doi/full/10.1056/NEJMoa2034577

5) https://www.nejm.org/doi/full/10.1056/nejmoa2035389

6) https://www.sciencedirect.com/science/article/pii/S0140673620326611

In [15]:
# Fixed
R0 = 2.85
_P_INFECT = None  # to calculate later 
P_REMOVE = 0.1
P_SYMPTOMS = 0.2
P_EXPOSED = 0.01

_RRR = [0.95, 0.67]

# Free
_P_VACCINATED = np.arange(0.001, 0.011, 0.001)
_P_VACCINATED_INITAL = np.arange(0, 1.1, 0.1)
_P_QUARANTINE = np.arange(0, 1.1, 0.1)

Note that we assume `P_EXPOSED` to be fixed. The reason for this is that the model is mostly invariant towards this value; changing it (for the most part) just moves the values horizontally. We denote parameters that need to be further realised/calculated for the actual model by prefixing them with an underscore `_`. `P_INFECT_S` can be calculated from knowing `R0`, `P_REMOVE`, as well as the mean and variance of the node degree distribution of the network (See [Dobson 2020](!https://pyepydemic.readthedocs.io/en/latest/cookbook/from-r-to-probabilities.html)).

In [16]:
def calc_p_infect(r0, p_remove, k_mean, k_var):
    """
    Calc p_infect.
    :param r0: basic reproduction number
    :param p_remove: probability of removal
    :param k_mean: Mean of node degree distribution
    :param k_var: Variance of node degree distribution
    :return: p_infect
    """
    return p_remove * r0 * (k_mean / (k_var - k_mean))

It also appears that asymptomatic cases (i.e. `exposed` compartment) are about 50% as infectious as symptomatic cases (`infected` compartment) (See [McEvoy et al. 2021](!https://bmjopen.bmj.com/content/11/5/e042354)). Hence, when required, we will set `P_INFECT_A` = `P_INFECT_S` $\times \ \frac{1}{2}$.

We can already set the fixed parameters for the models.

In [17]:
params_seir = dict()
params_seir[SEIR.P_SYMPTOMS] = P_SYMPTOMS
params_seir[SEIR.P_REMOVE] = P_REMOVE
params_seir[SEIR.P_EXPOSED] = P_EXPOSED

params_seivr = dict()
params_seivr[SEIVR.P_SYMPTOMS] = P_SYMPTOMS
params_seivr[SEIVR.P_REMOVE] = P_REMOVE
params_seivr[SEIVR.P_EXPOSED] = P_EXPOSED

## Simulations

We wrap some of the steps required to exectute and collect the results of an experiment in the `ExperimentRunner` class. We also require the epydemic `StochasticDynamics` and `Monitor`.

For all models we will also set a common maximum time of $150$ time steps. #todo review

In [18]:
from experiments.runner import ExperimentRunner
from epydemic import StochasticDynamics, Monitor

# define the monitor parameters
params_monitor = dict()
params_monitor[Monitor.DELTA] = 10

# save all simulation results in list of dataframes
simulation_dfs_seir = []
simulation_dfs_seivr = []

T = 150

### SEIR on MobilityNetwork

For each type of network we must first calculate the network-specific `P_INFECT`. In the case of mobility networks we can get the mean and variance of the power law cutoff distribution with the same parameters as that used for the node distribution of the network.

In [22]:
from model.distributions import PowerLawCutoffDist as PLCD
from epydemic.gf import gf_plc

tau = params_mobility_pre[MNG.EXPONENT]
kappa = params_mobility_pre[MNG.CUTOFF]
plcd = PLCD(tau, kappa)

P_INFECT_MOBILITY = calc_p_infect(R0, P_REMOVE, plcd.mean, plcd.var)

G0 = gf_plc(tau, kappa)
g0prime = G0.dx()

print(P_INFECT_MOBILITY, plcd.mean, g0prime(1))
del tau, kappa, plcd

0.0393142534049779 2.42305571610268 0.891122160071749


Having collected all parameters, we can now run the experiment using the `ExperimentRunner`.

In [23]:
# define the process
e = StochasticDynamics(MonitoredSEIR(), g=MNG())

# merge all parameters
params = params_mobility_pre | params_seir | params_monitor
params[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

er = ExperimentRunner(e)
er.run(params, T=T)

# er.plot('SEIR (PRE)', seir_compartments)
simulation_dfs_seir.append(er.to_df(seir_compartments, 'MN_pre', 'SEIR'))

del params, er, e

KeyboardInterrupt: 

In [20]:
# define the process
e = StochasticDynamics(MonitoredSEIR(), g=MNG())

# merge all parameters
params = params_mobility_post | params_seir | params_monitor
params[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

er = ExperimentRunner(e)
er.run(params, T=T)

simulation_dfs_seir.append(er.to_df(seir_compartments, 'MN_post', 'SEIR'))

del params, er, e

#### ... with quarantine

In [21]:
# define the process
e = StochasticDynamics(MonitoredSEIRQ(), g=MNG())

# merge all parameters
params = params_mobility_pre | params_seir | params_monitor
params[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params[SEIRQ.P_QUARANTINE] = 0.2

er = ExperimentRunner(e)
er.run(params, T=T)

simulation_dfs_seir.append(er.to_df(seir_compartments, 'MN_pre', 'SEIR_Q'))

del params, er, e

In [22]:
# define the process
e = StochasticDynamics(MonitoredSEIRQ(), g=MNG())

# merge all parameters
params = params_mobility_post | params_seir | params_monitor
params[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params[SEIRQ.P_QUARANTINE] = 0.2

er = ExperimentRunner(e)
er.run(params, T=T)

simulation_dfs_seir.append(er.to_df(seir_compartments, 'MN_post', 'SEIR_Q'))

del params, er, e

### SEIR on PLC

Again, we start by computing `R0` for the PLC network...

In [23]:
tau = params_plc_pre[PLCNetwork.EXPONENT]
kappa = params_plc_pre[PLCNetwork.CUTOFF]
plcd = PLCD(tau, kappa)

P_INFECT_PLC = calc_p_infect(R0, P_REMOVE, plcd.mean, plcd.var)
del tau, kappa, plcd

... and then run the simulations.

In [24]:
# define the process
e = StochasticDynamics(MonitoredSEIR(), g=PLCNetwork())

# merge all parameters
params = params_plc_pre | params_seir | params_monitor
params[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

er = ExperimentRunner(e)
er.run(params, T=T)

# er.plot('SEIR (PRE)', seir_compartments)
simulation_dfs_seir.append(er.to_df(seir_compartments, 'PLC_pre', 'SEIR'))

del params, er, e

#### ... with quarantine

In [25]:
# define the process
e = StochasticDynamics(MonitoredSEIRQ(), g=PLCNetwork())

# merge all parameters
params = params_plc_pre | params_seir | params_monitor
params[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params[SEIRQ.P_QUARANTINE] = 0.2

er = ExperimentRunner(e)
er.run(params, T=T)

simulation_dfs_seir.append(er.to_df(seir_compartments, 'PLC_pre', 'SEIR_Q'))

del params, er, e

### SEIR on Distanced

Calculate network based `R0`:

In [24]:
mean = DISTANCED_EXPONENT_PRE
var = DISTANCED_EXPONENT_PRE ** 2

P_INFECT_DISTANCED_PRE = calc_p_infect(R0, P_REMOVE, mean, var)
del mean, var

Run simulations:

In [25]:
# define the process
e = StochasticDynamics(MonitoredSEIR(), g=DNG())

# merge all parameters
params = params_distanced_pre | params_seir | params_monitor
params[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

er = ExperimentRunner(e)
er.run(params, T=T)

simulation_dfs_seir.append(er.to_df(seir_compartments, 'DISTANCED_pre', 'SEIR'))

del params, er, e

#### ... with quarantine

In [28]:
# define the process
e = StochasticDynamics(MonitoredSEIRQ(), g=DNG())

# merge all parameters
params = params_distanced_pre | params_seir | params_monitor
params[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params[SEIRQ.P_QUARANTINE] = 0.2

er = ExperimentRunner(e)
er.run(params, T=T)

simulation_dfs_seir.append(er.to_df(seir_compartments, 'DISTANCED_pre', 'SEIR_Q'))

del params, er, e

### SEIVR on Mobility

In [29]:
# define the process
e = StochasticDynamics(MonitoredSEIVR(), g=MNG())

# merge all parameters
params = params_mobility_pre | params_seivr | params_monitor
params[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params[SEIVR.P_VACCINATED] = 0.001
params[SEIVR.P_VACCINATED_INITIAL] = 0
params[SEIVR.VACCINE_RRR] = 0.67

er = ExperimentRunner(e)
er.run(params, T=T)

simulation_dfs_seivr.append(er.to_df(seivr_compartments, 'MN_pre', 'SEIVR'))

del params, er, e

#### ... with quarantine

In [30]:
# define the process
e = StochasticDynamics(MonitoredSEIVRQ(), g=MNG())

# merge all parameters
params = params_mobility_pre | params_seivr | params_monitor
params[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params[SEIVRQ.P_VACCINATED] = 0.001
params[SEIVRQ.P_VACCINATED_INITIAL] = 0
params[SEIVRQ.VACCINE_RRR] = 0.67
params[SEIVRQ.P_QUARANTINE] = 0.2

er = ExperimentRunner(e)
er.run(params, T=T)

simulation_dfs_seivr.append(er.to_df(seivr_compartments, 'MN_pre', 'SEIVR_Q'))

del params, er, e

### SEIVR on PLC

In [31]:
# define the process
e = StochasticDynamics(MonitoredSEIVR(), g=PLCNetwork())

# merge all parameters
params = params_plc_pre | params_seivr | params_monitor
params[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params[SEIVR.P_VACCINATED] = 0.001
params[SEIVR.P_VACCINATED_INITIAL] = 0
params[SEIVR.VACCINE_RRR] = 0.67

er = ExperimentRunner(e)
er.run(params, T=T)

simulation_dfs_seivr.append(er.to_df(seivr_compartments, 'PLC_pre', 'SEIVR'))

del params, er, e

#### ... with quarantine

In [32]:
# define the process
e = StochasticDynamics(MonitoredSEIVRQ(), g=PLCNetwork())

# merge all parameters
params = params_plc_pre | params_seivr | params_monitor
params[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params[SEIVRQ.P_VACCINATED] = 0.001
params[SEIVRQ.P_VACCINATED_INITIAL] = 0
params[SEIVRQ.VACCINE_RRR] = 0.67
params[SEIVRQ.P_QUARANTINE] = 0.2

er = ExperimentRunner(e)
er.run(params, T=T)

simulation_dfs_seivr.append(er.to_df(seivr_compartments, 'PLC_pre', 'SEIVR_Q'))

del params, er, e

### SEIVR on Distanced

In [33]:
# define the process
e = StochasticDynamics(MonitoredSEIVR(), g=DNG())

# merge all parameters
params = params_distanced_pre | params_seivr | params_monitor
params[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params[SEIVR.P_VACCINATED] = 0.001
params[SEIVR.P_VACCINATED_INITIAL] = 0
params[SEIVR.VACCINE_RRR] = 0.67

er = ExperimentRunner(e)
er.run(params, T=T)

simulation_dfs_seivr.append(er.to_df(seivr_compartments, 'DISTANCED_pre', 'SEIVR'))

del params, er, e

#### ... with quarantine

In [34]:
# define the process
e = StochasticDynamics(MonitoredSEIVRQ(), g=DNG())

# merge all parameters
params = params_distanced_pre | params_seivr | params_monitor
params[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params[SEIVRQ.P_VACCINATED] = 0.001
params[SEIVRQ.P_VACCINATED_INITIAL] = 0
params[SEIVRQ.VACCINE_RRR] = 0.67
params[SEIVRQ.P_QUARANTINE] = 0.2

er = ExperimentRunner(e)
er.run(params, T=T)

simulation_dfs_seivr.append(er.to_df(seivr_compartments, 'DISTANCED_pre', 'SEIVR_Q'))

del params, er, e

### Process simulation results

Appending all simulation results in a pandas data frame for SEIR and SEIVR will give us the data set containing all experimental results in long form.

In [35]:
from results.push_to_data_repo import update_or_create

# concatenate simulation data frames
sim_seir_df = pd.concat(simulation_dfs_seir)
sim_seivr_df = pd.concat(simulation_dfs_seivr)

# save to file
sim_seir_df.to_csv('results/sim_seir.csv')
sim_seivr_df.to_csv('results/sim_seivr.csv')

# push to repo
update_or_create('sim_seir.csv')
update_or_create('sim_seivr.csv')